In [ ]:
from pydub import AudioSegment
import numpy as np
import os
import librosa
from scipy.signal import argrelmax
import pandas as pd
from pydub import  silence
from pydub.playback import play
import matplotlib.pyplot as plt
from pydub.utils import mediainfo
import numpy, scipy.optimize
from scipy.signal import find_peaks

Reading the file:

In [ ]:
FILE_NAME='./sound_of_ball_original.wav'
info = mediainfo(FILE_NAME)
original_sample_rate=int(info['sample_rate'])
sound = AudioSegment.from_file(FILE_NAME)

In [ ]:
def speed_change(sound, speed=1.0):
    # Manually override the frame_rate. This tells the computer how many
    # samples to play per second
    sound_with_altered_frame_rate = sound._spawn(sound.raw_data, overrides={
        "frame_rate": int(sound.frame_rate * speed)
    })

    # convert the sound with altered frame rate to a standard frame rate
    # so that regular playback programs will work right. They often only
    # know how to play audio at standard frame rate (like 44.1k)
    return sound_with_altered_frame_rate.set_frame_rate(sound.frame_rate)

- Get the sample
- Plot it
- Get the most important frequency 
- Plot it low pass filtered sample
- Count the crosses

In [ ]:
samples = sound.get_array_of_samples()
samples_list=samples.tolist()
one_sample=samples[100000:180000]
short_sound = AudioSegment(one_sample.tobytes(), frame_rate=sound.frame_rate,sample_width=sound.sample_width,channels=1)
file_handle = short_sound.export("short_sound_output.wav", format="wav")
plt.figure(figsize=[20,8])
plt.plot(one_sample)

In [ ]:
rosa_sound, sr=librosa.load(FILE_NAME, sr=int(original_sample_rate))
ft=np.fft.fft(rosa_sound)
ft_mag=np.abs(ft)
# plt.plot(ft_mag[:10000])
ft_mag_list=ft_mag.tolist()
np.argmax(ft_mag_list)

In [ ]:
temp_ft=np.fft.fft(samples)
temp_ft_abs=np.abs(temp_ft)
temp_ft_abs=temp_ft_abs[:len(temp_ft_abs)//2]
# plt.plot(temp_ft_abs)
max_index=np.argmax(temp_ft_abs)
len(temp_ft_abs)-max_index #3196
print(max_index)
print(max(temp_ft_abs))
print(temp_ft_abs[max_index])

In [ ]:
f_ratio=0.2
freq_x=np.linspace(0, original_sample_rate, len(ft_mag))
num_freq_bins=int(len(freq_x)* f_ratio)
#plt.plot(freq_x[:num_freq_bins], ft_mag[:num_freq_bins])
# plt.plot([x if x > 3000 else 0 for x in ft_mag[:2000]])

In [ ]:
new_short_lpf=short_sound.low_pass_filter(max_index)

In [ ]:
plt.figure(figsize=[200,20])
short_sound_list=short_sound.get_array_of_samples()
new_short_lpf_list=new_short_lpf.get_array_of_samples()
plt.plot(short_sound_list)
plt.plot(new_short_lpf_list,color='r')

Smoothing the one sample
idea 1
- keep increasing the smoothing window until peaks find opposite peaks in the function

In [89]:
# one_sample=one_sample[10000:-10000]
smooth_window=1000
for smooth_window in range(1000, 100000,1000):
    one_sample_series=pd.Series(one_sample)
    one_sample_smooth1=one_sample_series.rolling(smooth_window).mean()
    plt.figure(figsize=[18,8])
    plt.plot(one_sample)
    plt.plot(one_sample_smooth1, color='r')
    ppeaks, _ = find_peaks(one_sample_smooth1)
    npeaks, _ = find_peaks(-one_sample_smooth1)
    all_peaks=list(ppeaks)+list(npeaks)
    all_peaks.sort()
    true_peaks=[]
    i=1
    for i in range(1,len(true_peaks)-1):
        if np.sign(one_sample_smooth1[all_peaks[i]])!= np.sign(one_sample_smooth1[all_peaks[i-1]]) and np.sign(one_sample_smooth1[all_peaks[i]])!= np.sign(one_sample_smooth1[all_peaks[i+1]]) :
            true_peaks.append(all_peaks[i])
    print(smooth_window, len(true_peaks))
# plt.plot(true_peaks, one_sample_smooth1[true_peaks], 'o', color='green')

1000 0
2000 0
3000 0
4000 0
5000 0
6000 0
7000 0
8000 0
9000 0
10000 0
11000 0
12000 0
13000 0
14000 0
15000 0
16000 0
17000 0
18000 0
19000 0
20000 0
21000 0
22000 0


/var/folders/g7/d67l17rx3kj9tcws82pm0d8c0000gn/T/ipykernel_22451/1473135771.py:6: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=[18,8])


23000 0
24000 0
25000 0
26000 0
27000 0
28000 0
29000 0
30000 0
31000 0
32000 0
33000 0
34000 0
35000 0
36000 0
37000 0
38000 0
39000 0
40000 0
41000 0
42000 0
43000 0
44000 0
45000 0
46000 0
47000 0
48000 0
49000 0
50000 0
51000 0
52000 0
53000 0
54000 0
55000 0
56000 0
57000 0
58000 0
59000 0
60000 0
61000 0
62000 0
63000 0
64000 0
65000 0
66000 0
67000 0
68000 0
69000 0
70000 0
71000 0
72000 0
73000 0
74000 0
75000 0
76000 0
77000 0
78000 0
79000 0
80000 0
81000 0
82000 0
83000 0
84000 0
85000 0
86000 0
87000 0
88000 0
89000 0
90000 0
91000 0
92000 0
93000 0
94000 0
95000 0
96000 0
97000 0
98000 0
99000 0


In [85]:
len(true_peaks)

0

Curve fitting

In [ ]:
from scipy import optimize
def test_func(x, a, b):
    return a * np.sin(b * x)
x_vals=np.linspace(0,len(samples)-1, num=len(samples))
params, params_covariance = optimize.curve_fit(test_func, x_vals, samples)
print(params)

In [ ]:
plt.figure(figsize=(16, 8))
plt.plot(x_vals[:len(one_sample)], one_sample, label='Data')
test_ddd=test_func(x_vals[:len(one_sample)], params[0], params[1])
plt.plot(x_vals[:len(one_sample)], test_ddd,
         label='Fitted function', color='r')
plt.legend(loc='best')
plt.show()

In [ ]:
plt.plot(test_ddd[1:100])

In [ ]:


powz=3
short_sound_list_powered=list(map(lambda x:pow(x,powz),short_sound_list))
short_sound_df=pd.Series(short_sound_list_powered)
short_sound_list_powered_smooth1=short_sound_df.rolling(500).mean()
plt.plot(short_sound_list_powered_smooth1,'+',color='g')
short_sound_list_powered_smooth2=short_sound_list_powered_smooth1.rolling(500).mean()
plt.plot(short_sound_list_powered_smooth1,'+',color='black')

# %%

slow_sound_lpf = speed_change(new_short_lpf, 0.1)
# play(slow_sound+20)
# %%
plt.figure(figsize=(12,7))
slow_sound_lpf_list=list(slow_sound_lpf.get_array_of_samples())
plt.plot(slow_sound)
# %%
file_handle = slow_sound_lpf.export("slow_sound_output.wav", format="wav")
# %%
slow_sound_list_df=pd.Series(slow_sound_lpf_list)
smooth1_slow_sound_list=slow_sound_list_df.rolling(10000).max()
smooth2_slow_sound_list=slow_sound_list_df.rolling(10000).min()
plt.figure(figsize=[15,5])
plt.plot(smooth1_slow_sound_list)
plt.plot(smooth2_slow_sound_list)
# plt.plot(slow_sound_list)
plt.show()

# %%
# peaks, _ = find_peaks(slow_sound_lpf_list, height=threshold, distance=min_distance_between_peaks)
maxAmp=max(slow_sound_list_df)
peaks, _ = find_peaks(slow_sound_lpf_list, height=0.5*maxAmp)
# %%
len(peaks)
# %%
# a positive peak is max between two negatives
peakpeak=[0]*len(slow_sound_list_df)
positives=[]
negatives=[]
for i in peaks:
    if slow_sound_list_df[i]<0:
        negatives.append((i, slow_sound_list_df[i]))
        # mark the peak
        if positives:
            aPair=max(positives, key=lambda item: item[1])
            print(aPair)
            peakpeak[aPair[0]]=aPair[1]
        positives=[]
    else:
        positives.append((i, slow_sound_list_df[i]))
        #negs
        if negatives:
            aPair=min(negatives, key=lambda item: item[1])
            peakpeak[aPair[0]]=aPair[1]
        negatives=[]
plt.figure(figsize=(20,7))
plt.plot(slow_sound_lpf_list)
plt.plot(peaks,slow_sound_list_df[peaks],'o', color='g')
plt.plot(peakpeak,'o',color='r')
sum([1 if i>0 else 0 for i in peakpeak])


# %%
peaks
# %% fitting sine wave

